In [12]:
import multiprocessing
num_cpu = multiprocessing.cpu_count()
num_cpu

8

In [13]:
# For Dataset
import os
import glob
import json
import time
import torch
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from facenet_pytorch import MTCNN, InceptionResnetV1

# For Model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import ZeroPadding2D, Convolution2D, MaxPooling2D, Dropout, Flatten, Activation, Dense
from tensorflow.keras.optimizers import SGD

# Resizing for tf shape
import einops

# Save result for later use
from numpy import asarray
from numpy import save

#from keras.models import load_model


In [14]:
# model_faces for detected faces

model_faces = keras.Sequential()
model_faces.add(ZeroPadding2D((1,1),input_shape=(82,82,64)))
model_faces.add(Convolution2D(64, (64, 64), activation='relu'))
model_faces.add(ZeroPadding2D((1,1)))
model_faces.add(Convolution2D(64, (3, 3), activation='relu'))
model_faces.add(MaxPooling2D((2,2), strides=(2,2)))

model_faces.add(ZeroPadding2D((1,1)))
model_faces.add(Convolution2D(128, (3, 3), activation='relu'))
model_faces.add(ZeroPadding2D((1,1)))
model_faces.add(Convolution2D(128, (3, 3), activation='relu'))
model_faces.add(MaxPooling2D((2,2), strides=(2,2)))

model_faces.add(Convolution2D(1024, (5, 5), activation='relu'))
model_faces.add(Dropout(0.5))
model_faces.add(Convolution2D(1024, (1, 1), activation='relu'))
model_faces.add(Dropout(0.5))
model_faces.add(Convolution2D(2622, (1, 1)))
model_faces.add(Flatten())
model_faces.add(Dense(1, activation='sigmoid'))

model_faces.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_4 (ZeroPaddin (None, 84, 84, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 21, 21, 64)        16777280  
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 21, 21, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
zero_padding2d_6 (ZeroPaddin (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 10, 10, 128)      

In [15]:
result = open('X-200', 'rb')
X = np.load(result, allow_pickle=True)

result2 = open('Y-200', 'rb')
y = np.load(result2, allow_pickle=True)

In [16]:
print(X.shape, X.dtype)
print(y.shape, y.dtype)

(3091, 82, 82, 64) float32
(3091,) int64


In [17]:
# 80:20 for split size
size = int(len(X) * 0.8)

x_train, x_test = X[:size], X[size:]
y_train, y_test = y[:size], y[size:]

In [18]:
# To clear memory
import gc
#del model_frames
del X
del y
gc.collect()

1432

In [19]:
model_faces.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = model_faces.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=10,
    use_multiprocessing=True,
    workers=num_cpu
)

Epoch 1/10
 3/20 [===>..........................] - ETA: 9:22 - loss: 0.6305 - accuracy: 0.7161

In [11]:
# Export model into a file
model_faces.save('model.h5')
#del model